# 할 일
- latex 형식 예쁘게 출력되게 하기
- 각 행별로 정보 출력하기(으악 디자인해뒀던 종이를 집에 두고와서... 오늘 집 가서 찬찬히 생각해보면서 하기) -> JOIN 꼭 사용할 수 있도록 해보기!!

In [15]:
import tkinter as tk
from tkinter import ttk
import mysql.connector


class ChapterApp:
    def __init__(self, root):
        self.root = root
        self.root.title("단원 선택")
        self._set_fullscreen()

        # Variables to store selected options
        self.selected_school = tk.StringVar()
        self.selected_grade = tk.StringVar()
        self.selected_semester = tk.StringVar()
        self.selected_chapter_lv1 = tk.StringVar()
        self.selected_chapter_lv2 = tk.StringVar()
        self.selected_chapter_lv3 = tk.StringVar()

        # Database connection
        self.db_connection = self.connect_to_db()

        # Main container frame
        self.container = ttk.Frame(self.root)
        self.container.pack(fill='both', expand=True)

        # Create and display main page
        self.create_main_page()

    def _set_fullscreen(self):
        """Set the window to fullscreen mode based on the platform."""
        import platform
        os_name = platform.system()
        
        if os_name == "Windows" or os_name == "Linux":
            # Windows/Linux: maximize the window
            try:
                self.root.state("zoomed")
            except Exception as e:
                print(f"Zoomed state failed: {e}")
                self.root.attributes("-fullscreen", True)
        elif os_name == "Darwin":  # macOS
            # macOS: use fullscreen attributes
            self.root.attributes("-fullscreen", True)
            # Adjust menubar visibility
            self.root.attributes("-topmost", True)

    def connect_to_db(self):
        try:
            connection = mysql.connector.connect(
                host="localhost",
                user="root",
                passwd="5749",
                database="math"
            )
            print("Database connected successfully!")
            return connection
        except mysql.connector.Error as err:
            print(f"Error: {err}")
            return None

    def create_main_page(self):
        # Main Page Layout
        for widget in self.container.winfo_children():
            widget.destroy()

        ttk.Label(self.container, text="학교를 선택하세요", font=('Helvetica', 18)).pack(pady=30)

        schools = ["초등학교", "중학교", "고등학교"]
        for school in schools:
            ttk.Button(
                self.container,
                text=school,
                command=lambda s=school: self.school_selected(s)
            ).pack(pady=10, padx=100, fill='x')

    def school_selected(self, school):
        self.selected_school.set(school)
        self.create_grade_page()

    def create_grade_page(self):
        # Grade Page Layout
        for widget in self.container.winfo_children():
            widget.destroy()

        ttk.Label(self.container, text="학년/학기를 선택하세요", font=('Helvetica', 18)).pack(pady=20)

        grade_frame = ttk.Frame(self.container)
        grade_frame.pack(pady=20)

        grades = range(1, 7) if self.selected_school.get() == "초등학교" else range(1, 4)
        for grade in grades:
            for semester in [1, 2]:
                btn_text = f"{grade}학년 {semester}학기"
                ttk.Button(
                    grade_frame,
                    text=btn_text,
                    command=lambda g=grade, s=semester: self.grade_selected(g, s)
                ).grid(row=grade-1, column=semester-1, padx=10, pady=10)

        ttk.Button(
            self.container,
            text="뒤로가기",
            command=self.create_main_page
        ).pack(pady=20)

    def grade_selected(self, grade, semester):
        def convert_grade(self, grade):
            if self.selected_school.get() == "초등학교":
                return grade
            elif self.selected_school.get() == "중학교":
                return grade + 6
            elif self.selected_school.get() == "고등학교":
                return grade + 9

        grade = convert_grade(self, grade)
        self.selected_grade.set(str(grade))
        self.selected_semester.set(str(semester))
        self.create_chapter_page()

    def create_chapter_page(self):
        # Chapter Page Layout
        for widget in self.container.winfo_children():
            widget.destroy()

        ttk.Label(self.container, text="단원을 선택하세요", font=('Helvetica', 18)).pack(pady=20)

        ttk.Label(self.container, text="대단원", font=('Helvetica', 14)).pack(pady=5)
        chapter_lv1_combo = ttk.Combobox(self.container, textvariable=self.selected_chapter_lv1, state="readonly")
        chapter_lv1_combo['values'] = self.get_chapter_lv1_options()
        chapter_lv1_combo.pack(pady=10)
        chapter_lv1_combo.bind("<<ComboboxSelected>>", lambda e: self.update_chapter_lv2_options())

        ttk.Label(self.container, text="중단원", font=('Helvetica', 14)).pack(pady=5)
        self.chapter_lv2_combo = ttk.Combobox(self.container, textvariable=self.selected_chapter_lv2, state="readonly")
        self.chapter_lv2_combo.pack(pady=10)
        self.chapter_lv2_combo.bind("<<ComboboxSelected>>", lambda e: self.update_chapter_lv3_options())

        ttk.Label(self.container, text="소단원", font=('Helvetica', 14)).pack(pady=5)
        self.chapter_lv3_combo = ttk.Combobox(self.container, textvariable=self.selected_chapter_lv3, state="readonly")
        self.chapter_lv3_combo.pack(pady=10)

        ttk.Button(
            self.container,
            text="확인",
            command=self.show_results
        ).pack(pady=20)

        ttk.Button(
            self.container,
            text="뒤로가기",
            command=self.create_grade_page
        ).pack(pady=10)

    def get_chapter_lv1_options(self):
        query = "SELECT DISTINCT chapter_lv1 FROM knowledge WHERE course_grade = %s AND course_semester = %s"
        cursor = self.db_connection.cursor()
        cursor.execute(query, (self.selected_grade.get(), self.selected_semester.get()))
        return [row[0] for row in cursor.fetchall()]

    def update_chapter_lv2_options(self):
        chapter_lv1 = self.selected_chapter_lv1.get()
        query = "SELECT DISTINCT chapter_lv2 FROM knowledge WHERE chapter_lv1 = %s"
        cursor = self.db_connection.cursor()
        cursor.execute(query, (chapter_lv1,))
        self.chapter_lv2_combo['values'] = [row[0] for row in cursor.fetchall()]

    def update_chapter_lv3_options(self):
        chapter_lv2 = self.selected_chapter_lv2.get()
        query = "SELECT DISTINCT chapter_lv3 FROM knowledge WHERE chapter_lv2 = %s"
        cursor = self.db_connection.cursor()
        cursor.execute(query, (chapter_lv2,))
        self.chapter_lv3_combo['values'] = [row[0] for row in cursor.fetchall()]

    def show_results(self):
        selected_lv1 = self.selected_chapter_lv1.get()
        selected_lv2 = self.selected_chapter_lv2.get()
        selected_lv3 = self.selected_chapter_lv3.get()
        results_window = tk.Toplevel(self.root)
        results_window.title("단원 결과")
        ResultsPage(results_window, self.db_connection, selected_lv1, selected_lv2, selected_lv3)


class ResultsPage:
    def __init__(self, root, db_connection, chapter_lv1, chapter_lv2, chapter_lv3):
        self.root = root
        self.db_connection = db_connection
        self.chapter_lv1 = chapter_lv1
        self.chapter_lv2 = chapter_lv2
        self.chapter_lv3 = chapter_lv3
        self.create_results_page()

    def create_results_page(self):
        ttk.Label(self.root, text=f"단원: {self.chapter_lv1} -> {self.chapter_lv2} -> {self.chapter_lv3}", font=('Helvetica', 16)).pack(pady=20)
        
        query = """
            SELECT `name`, GROUP_CONCAT(`achievement.name` SEPARATOR '\n') 
            FROM knowledge 
            WHERE chapter_lv1 = %s AND chapter_lv2 = %s AND chapter_lv3 = %s
            GROUP BY `name`
        """
        cursor = self.db_connection.cursor()
        cursor.execute(query, (self.chapter_lv1, self.chapter_lv2, self.chapter_lv3))
        results = cursor.fetchall()

        for name, achievements in results:
            ttk.Label(self.root, text=f"단원명: {name}", font=('Helvetica', 14)).pack(pady=5)
            ttk.Label(self.root, text=f"학습목표: {achievements}", font=('Helvetica', 12)).pack(pady=5)


if __name__ == "__main__":
    root = tk.Tk()
    app = ChapterApp(root)
    root.mainloop()


Database connected successfully!
